# Training program for Experiment

## setting path

In [1]:
import sys
import os

sys.path.append(os.path.split(os.getcwd())[0])


## データプレプロセス

## vocablary作成
すでにvocab.txtが作成済みである場合不用

In [2]:
from fast_jtnn.mol_tree import MolTree
from getpass import getpass,getuser

import mysql
from mysql import connector
import warnings

# SQL server profile
host = "localhost"
user = None
passwd = None
port = 3306
database="chemoinfo"

# 
VOCAB_FILE = "./MS_vocab.txt"

# get massbank data from SQL server
try:
    if not isinstance(user,str):
        user = raw_input("user")
    if not isinstance(passwd,str):
        passwd = getpass()
    connect = connector.connect(host=host,user=user,password=passwd,port=port,database=database)
    cursor = connect.cursor()
    cursor.execute("""select smiles from massbank where ms_type="MS" and instrument_type="EI-B" and smiles<>'N/A'; """)
    smiles_list = cursor.fetchall()
except mysql.connector.Error as e:
    print("Something went wrong: {}".format(e))
    sys.exit(1)
finally:
    if passwd : del passwd
    if connect: connect.close()
    if cursor: cursor.close()

# create vocablary
succes = 0
fault = 0
cset = set()
for one in smiles_list:
    try:
        mol = MolTree(one[0])
    except AttributeError as e:
        warnings.warn("Entered An SMILES that does not meet the rules")
        continue
    for c in mol.nodes:
        cset.add(c.smiles)

# write vocab
with open(VOCAB_FILE,"w") as f:
    for one in cset:
        f.write(one+"\n")

user a
 ·


Something went wrong: 1045 (28000): Access denied for user 'a'@'localhost' (using password: YES)


NameError: name 'connect' is not defined

! mkdir vae_model/
%run ../fast_molvae/vae_train.py --train processed --vocab ./MS_vocab.txt --save_dir vae_model/

# Vocab,datasetのロード

In [2]:
from fast_jtnn import *
from MS_PredictModel import MS_Dataset,MS_Dataset_pickle,dataset_load
import pickle
import torch

VOCAB_FILE = "./MS_vocab.txt"

vocab = [x.strip("\r\n ") for x in open(VOCAB_FILE,"r")]
vocab = Vocab(vocab)

'''
MS_Dataset.QUERY = """select smiles,file_path from massbank where ms_type="MS" and instrument_type="EI-B" and smiles<>'N/A';"""
dataset = MS_Dataset(vocab=vocab,host="localhost",database="chemoinfo",batch_size=20)
'''
train_vali_rate = 0.9

train_dataset, vali_dataset = dataset_load("./massbank.pkl",vocab,20,train_vali_rate)
print("number of train dataset :",len(train_dataset))
print("number of validation dataset :",len(vali_dataset))

('number of train dataset :', 6716)
('number of validation dataset :', 747)


## モデルの作成

In [3]:
from ms_encoder import ms_peak_encoder,ms_peak_encoder_lstm
import torch.nn as nn
import torch
hidden_size = 100
latent_size = 56
depthT = 20
depthG = 3

dec_model = JTNNVAE(vocab, hidden_size, latent_size, depthT, depthG).to('cuda')
print dec_model
enc_model = ms_peak_encoder_lstm(train_dataset.max_spectrum_size,output_size=latent_size,hidden_size=100,embedding_size=10).to('cuda')
print enc_model

for param in dec_model.parameters():
    if param.dim() == 1:
        nn.init.constant_(param, 0)
    else:
        nn.init.xavier_normal_(param)
load_model = "./vae_model/model.iter-70000"
dec_model.load_state_dict(torch.load(load_model,map_location='cuda'))

print "Model #Params: %dK" % (sum([x.nelement() for x in dec_model.parameters()]) / 1000,)
print "Model #Params: %dK" % (sum([x.nelement() for x in enc_model.parameters()]) / 1000,)

/usr/local/anaconda-aisiars/envs/jtvae/lib/python2.7/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


JTNNVAE(
  (jtnn): JTNNEncoder(
    (embedding): Embedding(365, 100)
    (outputNN): Sequential(
      (0): Linear(in_features=200, out_features=100, bias=True)
      (1): ReLU()
    )
    (GRU): GraphGRU(
      (W_z): Linear(in_features=200, out_features=100, bias=True)
      (W_r): Linear(in_features=100, out_features=100, bias=False)
      (U_r): Linear(in_features=100, out_features=100, bias=True)
      (W_h): Linear(in_features=200, out_features=100, bias=True)
    )
  )
  (decoder): JTNNDecoder(
    (embedding): Embedding(365, 100)
    (W_z): Linear(in_features=200, out_features=100, bias=True)
    (U_r): Linear(in_features=100, out_features=100, bias=False)
    (W_r): Linear(in_features=100, out_features=100, bias=True)
    (W_h): Linear(in_features=200, out_features=100, bias=True)
    (W): Linear(in_features=128, out_features=100, bias=True)
    (U): Linear(in_features=128, out_features=100, bias=True)
    (U_i): Linear(in_features=200, out_features=100, bias=True)
    (W_o): 

## setting optimizer

In [4]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

optimizer = optim.Adam(enc_model.parameters(), lr=1e-3)
#optimizer = optim.SGD(enc_model.parameters(),lr=100)
scheduler = lr_scheduler.ExponentialLR(optimizer, 0.9)
scheduler.step()

In [5]:
from MS_PredictModel import ms_peak_encoder,MS_Dataset
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np

pbar = None
train_dataset.batch_size = 10
vali_dataset.batch_size = 10
def training(max_epoch = 100):
    global pbar
    total_step = 0
    meters = np.zeros(3)
    vali_meters = np.zeros(3)
    with open("log2.csv","w") as f:
        f.write("epoch,iter.,word,topo,assm,vali word,vali topo,vali assm\n")
    for epoch in range(max_epoch):
        print("epoch : ",epoch)
        for batch in train_dataset:
            x_batch, x_jtenc_holder, x_mpn_holder, x_jtmpn_holder,x,y = batch
            total_step+=1
            #pbar.update(1)
            x = x.to('cuda')
            y = y.to('cuda')
            
            enc_model.zero_grad()
            dec_model.zero_grad()
            optimizer.zero_grad()
            
            h = enc_model(x,y)
            tree_vec = h[:,:h.shape[1]/2]
            mol_vec  = h[:,h.shape[1]/2:]
            _, x_tree_mess = dec_model.jtnn(*x_jtenc_holder)
            word_loss, topo_loss, word_acc, topo_acc = dec_model.decoder(x_batch,tree_vec)
            assm_loss, assm_acc = dec_model.assm(x_batch, x_jtmpn_holder, mol_vec , x_tree_mess)
            total_loss = word_loss+topo_loss+assm_loss
            total_loss.backward()
            optimizer.step()
            
            meters = meters + np.array([word_acc * 100, topo_acc * 100, assm_acc * 100])
            if total_step % 200 == 0:
                vali_total = 0
                for batch in vali_dataset:
                    x_batch, x_jtenc_holder, x_mpn_holder, x_jtmpn_holder,x,y = batch
                    x = x.to('cuda')
                    y = y.to('cuda')
                    with torch.no_grad():
                        h = enc_model(x,y)
                        tree_vec = h[:,:h.shape[1]/2]
                        mol_vec  = h[:,h.shape[1]/2:]
                        _, x_tree_mess = dec_model.jtnn(*x_jtenc_holder)
                        word_loss, topo_loss, word_acc, topo_acc = dec_model.decoder(x_batch,tree_vec)
                        assm_loss, assm_acc = dec_model.assm(x_batch, x_jtmpn_holder, mol_vec , x_tree_mess)
                        vali_meters = vali_meters + np.array([word_acc * 100, topo_acc * 100, assm_acc * 100])
                        vali_total += 1
                meters /= 200
                vali_meters /= vali_total
                print "[%d] , Word: %.2f, Topo: %.2f, Assm: %.2f vali_Word: %.2f, vali_Topo: %.2f, vali_assm: %.2f" % (total_step,meters[0], meters[1], meters[2],vali_meters[0],vali_meters[1],vali_meters[2])
                with open("log2.csv","a") as f:
                    f.write("%d,%d,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f\n" % (epoch,total_step,meters[0], meters[1], meters[2],vali_meters[0],vali_meters[1],vali_meters[2]))
                sys.stdout.flush()
                meters *= 0
                vali_meters *= 0
            if total_step % 200 == 0:
                torch.save(enc_model.state_dict(), "./enc_model" + "/model.iter-" + str(total_step))

#import pdb; pdb.set_trace()
try:
    #if pbar is None:
        #pbar = tqdm()
    training(100)
except RuntimeError as e:
    #if pbar is not None:
        #del pbar
    import traceback
    print(traceback.format_exc())
    #import pdb; pdb.set_trace()
    print(e)



('epoch : ', 0)


/usr/local/anaconda-aisiars/envs/jtvae/lib/python2.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/anaconda-aisiars/envs/jtvae/lib/python2.7/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[200] , Word: 45.25, Topo: 87.88, Assm: 88.72 vali_Word: 47.30, vali_Topo: 87.69, vali_assm: 89.51
[400] , Word: 45.52, Topo: 87.83, Assm: 88.36 vali_Word: 49.59, vali_Topo: 87.29, vali_assm: 89.68
[600] , Word: 46.19, Topo: 87.87, Assm: 89.03 vali_Word: 46.49, vali_Topo: 87.87, vali_assm: 90.10
('epoch : ', 1)
[800] , Word: 47.53, Topo: 87.99, Assm: 89.86 vali_Word: 48.67, vali_Topo: 87.94, vali_assm: 89.56
[1000] , Word: 47.14, Topo: 88.25, Assm: 89.56 vali_Word: 47.31, vali_Topo: 88.23, vali_assm: 91.40
[1200] , Word: 48.30, Topo: 87.77, Assm: 88.84 vali_Word: 50.42, vali_Topo: 88.30, vali_assm: 89.17
('epoch : ', 2)
[1400] , Word: 49.00, Topo: 88.19, Assm: 88.93 vali_Word: 49.97, vali_Topo: 88.59, vali_assm: 89.91
[1600] , Word: 50.25, Topo: 88.85, Assm: 89.96 vali_Word: 51.24, vali_Topo: 88.97, vali_assm: 89.78
[1800] , Word: 50.18, Topo: 88.70, Assm: 88.05 vali_Word: 52.78, vali_Topo: 88.95, vali_assm: 89.26
[2000] , Word: 51.13, Topo: 89.38, Assm: 89.24 vali_Word: 52.42, vali_To

In [13]:
def evaluation():
    with torch.no_grad():
        with open("evaluation.csv","w") as f:
            f.write("true,predict\n")
        for batch in vali_dataset:
            x_batch, x_jtenc_holder, x_mpn_holder, x_jtmpn_holder,x,y = batch
            x = x.to('cuda')
            y = y.to('cuda')
            
            h = enc_model(x,y)
            tree_vec = h[:,:h.shape[1]/2]
            mol_vec  = h[:,h.shape[1]/2:]
            for num in range(h.size()[0]):
                with open("evaluation.csv","a") as f:
                    f.write(x_batch[num].smiles+","+dec_model.decode(tree_vec[num].view(1,latent_size/2),mol_vec[num].view(1,latent_size/2),False)+"\n")
            

evaluation()

In [51]:
import os
from rdkit import Chem
from rdkit.Chem import Draw

def _re_smiles(smiles1,smiles2):
    #print(smiles1,smiles2)
    smiles1 = Chem.MolToSmiles(Chem.MolFromSmiles(smiles1),True)
    smiles2 = Chem.MolToSmiles(Chem.MolFromSmiles(smiles2),True)
    return smiles1 == smiles2

def is_structural_isomer(smiles1,smiles2):
    def Molecular_formula(smiles):
        atoms = {}
        mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
        for atom in mol.GetAtoms():
            if not atom.GetSymbol() in atoms:
                atoms[atom.GetSymbol()] = 1
            else:
                atoms[atom.GetSymbol()] += 1
        return atoms
    atoms1 = Molecular_formula(smiles1)
    atoms2 = Molecular_formula(smiles2)
    return atoms1 == atoms2
    
def analyze_result(list_name="evaluation.csv",path="image_list"):
    if not os.path.exists(path):
        os.mkdir(path)
    with open(list_name,"r") as f:
        tp_list = [one.strip().split(",") for one in f.readlines()]
    
    print "Number of data: %d"% (len(tp_list)-1)
        
    true_list = [[i,one[0]] for i,one in enumerate(tp_list[1:]) if _re_smiles(one[0],one[1])]
    print "Number of matching: %d" % (len(true_list))
    print(true_list)
    
    true_list = [[i,one[0]] for i,one in enumerate(tp_list[1:]) if is_structural_isomer(one[0],one[1])]
    print "Number of matching: %d" % (len(true_list))
    print(true_list)
        
        
    #for i,smiles in enumerate(tp_list[1:]):
    #    true_mol = Chem.MolFromSmiles(smiles[0])
    #    pred_mol = Chem.MolFromSmiles(smiles[1])
    #    Draw.MolToFile(true_mol,os.path.join(path,"%s.png" % smiles[0]))
    #    image = Draw.MolsToImage([true_mol,pred_mol])
    #    image.save(os.path.join(path,"%d.png" % i))
    
analyze_result()

Number of data: 740
Number of matching: 5
[[1, 'COCC(=O)OC'], [114, 'CC(C1)CCC(=O)C1'], [344, 'CC(C)CCOC(C)=O'], [450, 'CCCC(=O)c(c1)cccc1'], [575, 'COCC([H])(O1)C([H])(OC)C([H])(OC)C([H])(OC)C([H])(OC)1']]
Number of matching: 24
[[1, 'COCC(=O)OC'], [29, 'CC=CCO'], [47, 'CC(C)CCOC(=O)C(C)C'], [76, 'CCC=NN(CC)CC'], [78, 'CCCCC(CC)OC(=O)C(C)C'], [114, 'CC(C1)CCC(=O)C1'], [148, 'C=Cc(c1)cc(c2)c(ccc2)c1'], [159, 'CNc(c1)cccc1'], [168, 'Cc(c1)cc(N)cc(C)1'], [195, 'OC(=O)CCc(c1)cccc1'], [252, 'CCCCCOC(=O)c(c1)c(O)ccc1'], [343, 'COCCOC'], [344, 'CC(C)CCOC(C)=O'], [365, 'CCOC(=O)c(c1)cc(cc1)C(C)=O'], [393, 'CCOCCOCC'], [450, 'CCCC(=O)c(c1)cccc1'], [468, 'Cc(c1)ccc(C)c1'], [509, 'Cc(c1)c(C)c(C)cc1'], [539, 'CCCCC(CC)C(=O)OCC'], [575, 'COCC([H])(O1)C([H])(OC)C([H])(OC)C([H])(OC)C([H])(OC)1'], [616, 'CCCCCCCCCCOC(=O)CCC'], [651, 'Cc(c1)ccc(C)c1'], [688, 'Cc(c1)c(Br)ccc1'], [717, 'COC(=O)C(C)O']]


In [36]:
len(tp_list)

NameError: name 'tp_list' is not defined